In [1]:
import streamlit as st
import mysql.connector
import pandas as pd
import numpy as np
import random
import time
import plotly.express as px

# Define the list of features
feature_columns = [
    'set1',
    'set2',
    'sensor2',
    'sensor3',
    'sensor4',
    'sensor6',
    'sensor7',
    'sensor8',
    'sensor9',
    'sensor11',
    'sensor12',
    'sensor13',
    'sensor14',
    'sensor15',
    'sensor17',
    'sensor20',
    'sensor21'
]

# Define the min, max, mean values for each feature based on your statistics.
feature_stats = {
    'set1':    {'min': -0.0087, 'max': 0.0087,   'mean': -0.000009},
    'set2':    {'min': -0.0006, 'max': 0.0006,   'mean': 0.000002},
    'sensor2': {'min': 641.21,  'max': 644.53,   'mean': 642.680934},
    'sensor3': {'min': 1571.04, 'max': 1616.91,  'mean': 1590.523119},
    'sensor4': {'min': 1382.25, 'max': 1441.49,  'mean': 1408.933782},
    'sensor6': {'min': 21.60,   'max': 21.61,    'mean': 21.609803},
    'sensor7': {'min': 549.85,  'max': 556.06,   'mean': 553.367711},
    'sensor8': {'min': 2387.90, 'max': 2388.56,  'mean': 2388.096652},
    'sensor9': {'min': 9021.73, 'max': 9244.59,  'mean': 9065.242941},
    'sensor11':{'min': 46.85,   'max': 48.53,    'mean': 47.541168},
    'sensor12':{'min': 518.69,  'max': 523.38,   'mean': 521.413470},
    'sensor13':{'min': 2387.88, 'max': 2388.56,  'mean': 2388.096152},
    'sensor14':{'min': 8099.94, 'max': 8293.72,  'mean': 8143.752722},
    'sensor15':{'min': 8.3249,  'max': 8.5848,   'mean': 8.442146},
    'sensor17':{'min': 388.0,   'max': 400.0,    'mean': 393.210654},
    'sensor20':{'min': 38.14,   'max': 39.43,    'mean': 38.816271},
    'sensor21':{'min': 22.8942, 'max': 23.6184,  'mean': 23.289705}
}

# ===== MySQL Database Setup =====
# Update these connection parameters to match your MySQL server settings.
db_connection = mysql.connector.connect(
    host='localhost',
    user='root',      # Replace with your MySQL username
    password='9998',  # Replace with your MySQL password
    database='ml'   # Replace with your MySQL database name
)
cursor = db_connection.cursor()

# Create the sensor_data table if it doesn't exist.
create_table_query = f"""
CREATE TABLE IF NOT EXISTS sensor_data (
    id INT AUTO_INCREMENT PRIMARY KEY,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    {', '.join([f"{col} FLOAT" for col in feature_columns])}
)
"""
cursor.execute(create_table_query)
db_connection.commit()


In [2]:
# ===== Functions =====
def insert_sensor_data(data_dict):
    """Insert sensor readings into the database."""
    query = f"INSERT INTO sensor_data ({', '.join(feature_columns)}) VALUES ({', '.join(['%s']*len(feature_columns))})"
    values = tuple(data_dict[col] for col in feature_columns)
    cursor.execute(query, values)
    db_connection.commit()

def fetch_last_10():
    """Fetch the last 10 rows of sensor data (most recent first, then reverse order)."""
    query = f"SELECT id, timestamp, {', '.join(feature_columns)} FROM sensor_data ORDER BY id DESC LIMIT 10"
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = ['id', 'timestamp'] + feature_columns
    df = pd.DataFrame(rows, columns=columns)
    return df.iloc[::-1]  # Reverse to have the earliest row first

def pad_data(df, target_rows=10):
    """If there are fewer than 10 rows, repeat the last available row to pad the data."""
    if len(df) < target_rows and not df.empty:
        last_row = df.iloc[-1:]
        pad_count = target_rows - len(df)
        df = pd.concat([df, pd.concat([last_row]*pad_count, ignore_index=True)], ignore_index=True)
    return df

def predict_rul(data):
    """
    Dummy prediction function.
    Replace this with your LSTM model's prediction code.
    Here, we compute the average of all feature values (over all rows) and scale it by a random factor.
    """
    avg_value = data[feature_columns].mean().mean()
    prediction = avg_value * random.uniform(0.8, 1.2)
    return prediction

def generate_sensor_data():
    """
    Generate sensor readings for each feature using the defined min and max.
    (You can modify this function to use other distributions if desired.)
    """
    return {col: random.uniform(feature_stats[col]['min'], feature_stats[col]['max'])
            for col in feature_columns}

In [6]:
sensor_data = generate_sensor_data()

In [7]:
sensor_data

{'set1': 0.006392289605085824,
 'set2': -0.0005504720457706677,
 'sensor2': 644.4217770165271,
 'sensor3': 1594.0470581039326,
 'sensor4': 1384.048584431665,
 'sensor6': 21.603610869222077,
 'sensor7': 555.1902898398,
 'sensor8': 2388.0016779301745,
 'sensor9': 9211.046010150158,
 'sensor11': 48.48644267577196,
 'sensor12': 519.698701732227,
 'sensor13': 2388.0542409059913,
 'sensor14': 8225.04850630316,
 'sensor15': 8.576256869507244,
 'sensor17': 393.0288114299379,
 'sensor20': 38.52856539683007,
 'sensor21': 23.498445479980777}

In [8]:
insert_sensor_data(sensor_data)

In [9]:
df_data = fetch_last_10()
df_data

,id,timestamp,set1,set2,sensor2,sensor3,sensor4,sensor6,sensor7,sensor8,sensor9,sensor11,sensor12,sensor13,sensor14,sensor15,sensor17,sensor20,sensor21
1,1,2025-03-02 14:18:09,-0.005242,0.000376,643.570,1603.83,1425.08,21.6089,550.168,2388.27,9206.53,47.4826,519.249,2387.95,8108.16,8.41788,389.936,38.2922,22.9845
0,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984


In [10]:
df_data = pad_data(df_data, 10)
df_data

,id,timestamp,set1,set2,sensor2,sensor3,sensor4,sensor6,sensor7,sensor8,sensor9,sensor11,sensor12,sensor13,sensor14,sensor15,sensor17,sensor20,sensor21
0,1,2025-03-02 14:18:09,-0.005242,0.000376,643.570,1603.83,1425.08,21.6089,550.168,2388.27,9206.53,47.4826,519.249,2387.95,8108.16,8.41788,389.936,38.2922,22.9845
1,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984
2,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984
3,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984
4,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984
5,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984
6,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984
7,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984
8,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984
9,2,2025-03-02 14:18:20,0.006392,-0.000550,644.422,1594.05,1384.05,21.6036,555.190,2388.00,9211.05,48.4864,519.699,2388.05,8225.05,8.57626,393.029,38.5286,23.4984


In [43]:
tem = df_data[feature_columns].values

In [44]:
tem.shape

(10, 17)

In [31]:
import joblib

In [32]:
model = joblib.load('model.pkl', 'rb')

In [33]:
model

<Sequential name=sequential, built=True>

In [34]:
scaler = joblib.load('scaler.pkl')

In [35]:
scaler

MinMaxScaler()

In [45]:
data = scaler.transform(tem)

C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [46]:
model.predict(data.reshape(1, 10, 17))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[8.505498]], dtype=float32)

In [50]:
import matplotlib.pyplot as plt

In [53]:
plt.plot(data = df_data, scalex='timestamp', scaley='sensor2',
                  title="Last 10 Sensor2 Readings",
                  markers=True)

[]